In [ ]:
%pip install pandas
%pip install requests

In [3]:

import os
import pandas as pd
import requests
import zipfile
from IPython.display import clear_output

In [4]:
# Variaveis Globais
diretorio_raiz = os.getcwd()
diretorio_dados_cvm = f"{diretorio_raiz}/dados_cvm"
diretorio_dados_empresas = f"{diretorio_raiz}/desmonstracoes_empresas"

# URL base para download dos dados
url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

# Definindo o intervalo de anos
anos = range(2010, 2024)

In [ ]:
# Cria um novo diretório chamado "dados_cvm" no diretório de trabalho atual
try:
    # Cria o diretório
    os.makedirs(diretorio_dados_cvm)

    # Lista o conteúdo do diretório
    os.listdir(diretorio_dados_cvm)

    # Imprime uma mensagem
    print("Diretório criado")

# Trata os erros
except OSError as e:
    # Imprime uma mensagem de erro se o diretório não puder ser criado
    print("Erro: Criando diretório. " + diretorio_dados_cvm)
    print(e)

In [ ]:
# Cria um novo diretório chamado "dados_empresas" no diretório de trabalho atual
try:
    # Cria o diretório

    os.makedirs(diretorio_dados_empresas)

    # Lista o conteúdo do diretório
    os.listdir(diretorio_dados_empresas)

    # Imprime uma mensagem
    print("Diretório criado")

# Trata os erros
except OSError as e:
    # Imprime uma mensagem de erro se o diretório não puder ser criado
    print("Erro: Criando diretório. " + diretorio_dados_empresas)
    print(e)

In [ ]:
# Muda o diretório de trabalho para o diretório criado
os.chdir(diretorio_dados_cvm)


# Loop para cada ano no intervalo
def downloadCVM(periodo: range, url: str):
    for ano in periodo:
        exist_file = os.path.exists(f"dfp_cia_aberta_{ano}.zip")

        if exist_file:
            print(f"Arquivo referente ao ano {ano} já existe")
        else:
            # Faz o download do arquivo .zip para o ano atual
            download = requests.get(url + f"dfp_cia_aberta_{ano}.zip")
            print(f"Fazendo download do arquivo referente ao ano: {ano}")
            with open(f"dfp_cia_aberta_{ano}.zip", "wb") as f:
                f.write(download.content)
                print(f"Download do arquivo referente ao ano: {ano} concluído")


downloadCVM(anos, url_base)

In [2]:

# Muda o diretório de trabalho para o diretório criado
os.chdir(diretorio_dados_cvm)

# listar arquivos
arquivos = os.listdir(diretorio_dados_cvm)


total_empresas = 0
# Itera sobre cada arquivo no diretório
for i_arquivos, arquivo in enumerate(arquivos, start=1):
    # Muda o diretório de trabalho para o diretório criado
    os.chdir(diretorio_dados_cvm)

    if not arquivo.endswith(".zip"):
        continue

    # Abre o arquivo zip
    arquivo_zip = zipfile.ZipFile(arquivo)

    lista_demonstracoes_ano = []

    # Itera sobre cada arquivo no arquivo zip
    for planilha_csv in arquivo_zip.namelist():
        if not planilha_csv.endswith(".csv"):
            continue

        # Lê o arquivo CSV e adiciona à lista de demonstrações
        demonstracao = pd.read_csv(arquivo_zip.open(
            planilha_csv), sep=";", encoding='ISO-8859-1', dtype={"ORDEM_EXERC": "category"})
        lista_demonstracoes_ano.append(demonstracao)

    # Concatena os DataFrames das demonstrações
    df_demonstracoes_ano = pd.concat(lista_demonstracoes_ano)

    # Processamento dos dados das demonstrações

    df_demonstracoes_ano[['CON_IND', 'TIPO_DEMO']
                         ] = df_demonstracoes_ano['GRUPO_DFP'].str.split("-", n=1, expand=True)
    # Remove a coluna ['GRUPO_DFP']
    df_demonstracoes_ano.drop('GRUPO_DFP', axis=1, inplace=True)
    df_demonstracoes_ano['CON_IND'] = df_demonstracoes_ano['CON_IND'].str.strip()
    df_demonstracoes_ano['TIPO_DEMO'] = df_demonstracoes_ano['TIPO_DEMO'].str.strip(
    )
    df_demonstracoes_ano = df_demonstracoes_ano[df_demonstracoes_ano['ORDEM_EXERC'] != "PENÚLTIMO"]

    # Muda o diretório de trabalho para o diretório de dados das empresas
    os.chdir(diretorio_dados_empresas)

    # Obtém as empresas únicas do ano
    empresas_ano = df_demonstracoes_ano["CNPJ_CIA"].unique()

    # Itera sobre cada empresa única
    for i_empresa_ano, empresa_ano in enumerate(empresas_ano, start=1):
        demonstracoes_empresa = df_demonstracoes_ano[df_demonstracoes_ano['CNPJ_CIA'] == empresa_ano]

        # Formata o nome da empresa
        nome_empresa = demonstracoes_empresa['DENOM_CIA'].unique()[0].replace(
            " ", "-").replace(".", "").replace("/", "").replace("---", "-")

        # Verifica se o arquivo da empresa existe
        if not os.path.exists(f'{nome_empresa}.csv'):
            with open(f'{nome_empresa}.csv', 'w') as f:
                f.write('')
            total_empresas += 1

        # Salva as demonstrações da empresa no arquivo CSV correspondente
        demonstracoes_empresa.to_csv(
            f'{nome_empresa}.csv', sep=";", mode='a',  index=False)
        print(
            f"Gerando demonstrações da empresa {nome_empresa}, faltam {len(empresas_ano) - i_empresa_ano} empresas e {len(arquivos) - i_arquivos} arquivos!")

        # Remove os dados da empresa do DataFrame original
        df_demonstracoes_ano = df_demonstracoes_ano[df_demonstracoes_ano["CNPJ_CIA"] != empresa_ano]
        clear_output()

    print(f"{len(empresas_ano)} empresas geradas pelo arquivo {arquivo}, faltam {len(empresas_ano) - i_empresa_ano} empresas e {len(arquivos) - i_arquivos} arquivos!")

clear_output()
print(f"Último arquivo {arquivo} lido, foram {len(arquivos)} arquivos!")
print(f"Foram {total_empresas} empresas geradas!")

Último arquivo dfp_cia_aberta_2023.zip lido foram 5 arquivos!
Foram 3591 empresas geradas!
